In [9]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import sys
from os import listdir
from os.path import isfile, join
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from tabulate import tabulate
from tqdm import tqdm

In [2]:
FEATURES_SET = {
    "feature": 1,
    "permission": 2,
    "activity": 3,
    "service_receiver": 3,
    "provider": 3,
    "service": 3,
    "intent": 4,
    "api_call": 5,
    "real_permission": 6,
    "call": 7,
    "url": 8
}


def count_feature_set(lines):
    """
    Count how many features belong to a specific set
    :param lines: features in the text file
    :return:
    """
    features_map = {x: 0 for x in range(1, 9)}
    for l in lines:
        if l != "\n":
            set = l.split("::")[0]
            features_map[FEATURES_SET[set]] += 1
    features = []
    for i in range(1, 9):
        features.append(features_map[i])
    return features


In [3]:
def read(LOAD_DATA=False):
    if LOAD_DATA:
        print("Previous data not loaded. Attempt to read data ...")
        mypath = r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton\Drebin\MetaData\feature_vectors\feature_vectors"
        onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

        print("Reading csv file for ground truth ...")
        ground_truth = np.loadtxt(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton\Drebin\MetaData\sha256_family.csv", delimiter=",", skiprows=1, dtype=str)
        # print ground_truth.shape
        # families = np.unique(ground_truth[:, 1])
        # print families
        # print len(families)

        print("Reading positive and negative texts ...")
        pos = []
        neg = []
        for virus in tqdm(onlyfiles):
            if virus in ground_truth[:, 0]:
                pos.append(virus)
            else:
                if len(neg) < 5560:
                #if len(neg) < 22240:
                    neg.append(virus)

        print("Extracting features ...")
        x = []
        y = []
        for text_file in tqdm(pos):
            sys.stdin = open("%s/%s" % (mypath, text_file))
            features = sys.stdin.readlines()
            sample = count_feature_set(features)
            x.append(sample)
            y.append(1)

        for text_file in tqdm(neg):
            sys.stdin = open("%s/%s" % (mypath, text_file))
            features = sys.stdin.readlines()
            sample = count_feature_set(features)
            x.append(sample)
            y.append(0)

        print("Data is read successfully:")
        x = np.array(x)
        y = np.array(y)
        print(x.shape, y.shape)

        print("Saving data under data_numpy directory ...")
        np.save(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/x_all.npy", x)
        np.save(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/y_all.npy", y)

        return x, y
    else:
        print("Loading previous data ...")
        x_ = np.load(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/x_all.npy")
        y_ = np.load(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/y_all.npy")
        print(x_.shape, y_.shape)
        # print x == x_, y == y_
        return x_, y_


def map_family_to_category(families):
    out = {}
    count = 1
    for family in families:
        out[family] = count
        count += 1
    return out


if __name__ == "__main__":
    x, y = read(LOAD_DATA=True)

Previous data not loaded. Attempt to read data ...
Reading csv file for ground truth ...
Reading positive and negative texts ...


100%|██████████| 129013/129013 [00:01<00:00, 70960.38it/s]


Extracting features ...


100%|██████████| 5560/5560 [00:25<00:00, 218.34it/s]

Data is read successfully:
(11120, 8) (11120,)
Saving data under data_numpy directory ...


In [4]:
x_all, y_all = read(LOAD_DATA=False)
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

Loading previous data ...
(11120, 8) (11120,)
(8896, 8) (8896,)
(2224, 8) (2224,)


In [ ]:
# # Contagem de classes antes da divisão
# y_all_counts = pd.Series(y_all).value_counts(normalize=True)

# # Divisão dos dados em treinamento e teste
# x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=42, stratify=y_all)

# # Contagem de classes após a divisão
# y_train_counts = pd.Series(y_train).value_counts(normalize=True)
# y_test_counts = pd.Series(y_test).value_counts(normalize=True)

# # Verificação se as proporções foram mantidas
# print("Proporções das classes no conjunto original:")
# print(y_all_counts)
# print("\nProporções das classes no conjunto de treinamento:")
# print(y_train_counts)
# print("\nProporções das classes no conjunto de teste:")
# print(y_test_counts)

In [6]:
####### DP = Desvio Padrão; PosBal = Pos Balanceamento
Acurracy = []
Precision = []
Recall = []
F1_score = []
Roc_auc = []
######
AcurracyDP = []
PrecisionDP = []
RecallDP = []
F1_scoreDP = []
Roc_aucDP = []
######
AcurracyBagging = []
PrecisionBagging = []
RecallBagging = []
F1Bagging = []
Roc_aucBagging = []
######
AcurracyBaggingDP = []
PrecisionBaggingDP = []
RecallBaggingDP = []
F1BaggingDP = []
Roc_aucBaggingDP = []
#####
AcurracyPosBal = []
PrecisionPosBal = []
RecallPosBal = []
F1_scorePosBal = []
Roc_aucPosBal = []
######
AcurracyDPPosBal = []
PrecisionDPPosBal = []
RecallDPPosBal = []
F1_scoreDPPosBal = []
Roc_aucDPPosBal = []
######
AcurracyBaggingPosBal = []
PrecisionBaggingPosBal = []
RecallBaggingPosBal = []
F1BaggingPosBal = []
Roc_aucBaggingPosBal = []
######
AcurracyBaggingDPPosBal = []
PrecisionBaggingDPPosBal = []
RecallBaggingDPPosBal = []
F1BaggingDPPosBal = []
Roc_aucBaggingDPPosBal = []

### Modelos Base

In [ ]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all)

    # Inicializar o modelo de árvore de decisão com Gini Impurity e sem limitação de altura
    modelo_arvore = DecisionTreeClassifier(criterion='gini')

    # Treinar o modelo no conjunto de treino
    modelo_arvore.fit(x_train, y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_arvore.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
Acurracy.append(media_acuracias)
Precision.append(media_precisoes)
Recall.append(media_recalls)
F1_score.append(media_f1_scores)
Roc_auc.append(media_roc_auc_scores)

AcurracyDP.append(dp_acuracias)
PrecisionDP.append(dp_precisoes)
RecallDP.append(dp_recalls)
F1_scoreDP.append(dp_f1_scores)
Roc_aucDP.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [00:00<00:08,  3.56it/s]

Repetição 1 - Acurácia: 0.98, Precisão: 0.77, Recall: 0.82, F1-score: 0.79, ROC AUC: 0.90


  7%|▋         | 2/30 [00:00<00:07,  3.90it/s]

Repetição 2 - Acurácia: 0.98, Precisão: 0.79, Recall: 0.81, F1-score: 0.80, ROC AUC: 0.90


 10%|█         | 3/30 [00:00<00:06,  3.99it/s]

...

Repetição 28 - Acurácia: 0.98, Precisão: 0.76, Recall: 0.82, F1-score: 0.79, ROC AUC: 0.91


 97%|█████████▋| 29/30 [00:07<00:00,  4.15it/s]

Repetição 29 - Acurácia: 0.98, Precisão: 0.75, Recall: 0.81, F1-score: 0.78, ROC AUC: 0.90


100%|██████████| 30/30 [00:07<00:00,  4.12it/s]

Repetição 30 - Acurácia: 0.98, Precisão: 0.76, Recall: 0.79, F1-score: 0.78, ROC AUC: 0.89
Média das Acurácias: 0.98179 ± 0.00079
Média das Precisões: 0.77154 ± 0.01227
Média dos Recalls: 0.82068 ± 0.00998
Média dos F1-scores: 0.79527 ± 0.00802
Média dos ROC AUC: 0.90486 ± 0.00493


In [7]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all)
    
    #Inicializar o modelo de Naive Bayes com distribuição de Bernoulli
    modelo_naive_bayes = BernoulliNB()

    # Treinar o modelo no conjunto de treino
    modelo_naive_bayes.fit(x_train, y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_naive_bayes.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
Acurracy.append(media_acuracias)
Precision.append(media_precisoes)
Recall.append(media_recalls)
F1_score.append(media_f1_scores)
Roc_auc.append(media_roc_auc_scores)

AcurracyDP.append(dp_acuracias)
PrecisionDP.append(dp_precisoes)
RecallDP.append(dp_recalls)
F1_scoreDP.append(dp_f1_scores)
Roc_aucDP.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

 67%|██████▋   | 20/30 [00:00<00:00, 95.70it/s]

Repetição 1 - Acurácia: 0.54, Precisão: 0.52, Recall: 0.92, F1-score: 0.67, ROC AUC: 0.54
Repetição 2 - Acurácia: 0.54, Precisão: 0.52, Recall: 0.92, F1-score: 0.67, ROC AUC: 0.54
Repetição 3 - Acurácia: 0.54, Precisão: 0.52, Recall: 0.93, F1-score: 0.67, ROC AUC: 0.54
Repetição 4 - Acurácia: 0.54, Precisão: 0.52, Recall: 0.92, F1-score: 0.67, ROC AUC: 0.54
Repetição 5 - Acurácia: 0.54, Precisão: 0.53, Recall: 0.92, F1-score: 0.67, ROC AUC: 0.54
Repetição 6 - Acurácia: 0.55, Precisão: 0.53, Recall: 0.94, F1-score: 0.68, ROC AUC: 0.55
Repetição 7 - Acurácia: 0.54, Precisão: 0.52, Recall: 0.93, F1-score: 0.67, ROC AUC: 0.54
Repetição 8 - Acurácia: 0.54, Precisão: 0.53, Recall: 0.93, F1-score: 0.67, ROC AUC: 0.54
Repetição 9 - Acurácia: 0.54, Precisão: 0.52, Recall: 0.93, F1-score: 0.67, ROC AUC: 0.54
Repetição 10 - Acurácia: 0.55, Precisão: 0.53, Recall: 0.93, F1-score: 0.67, ROC AUC: 0.55
Repetição 11 - Acurácia: 0.56, Precisão: 0.53, Recall: 0.93, F1-score: 0.68, ROC AUC: 0.56
Repetiçã

100%|██████████| 30/30 [00:00<00:00, 88.90it/s]

Repetição 21 - Acurácia: 0.55, Precisão: 0.53, Recall: 0.94, F1-score: 0.68, ROC AUC: 0.55
Repetição 22 - Acurácia: 0.54, Precisão: 0.52, Recall: 0.92, F1-score: 0.67, ROC AUC: 0.54
Repetição 23 - Acurácia: 0.55, Precisão: 0.53, Recall: 0.92, F1-score: 0.67, ROC AUC: 0.55
Repetição 24 - Acurácia: 0.54, Precisão: 0.53, Recall: 0.93, F1-score: 0.67, ROC AUC: 0.54
Repetição 25 - Acurácia: 0.54, Precisão: 0.52, Recall: 0.92, F1-score: 0.67, ROC AUC: 0.54
Repetição 26 - Acurácia: 0.54, Precisão: 0.53, Recall: 0.93, F1-score: 0.67, ROC AUC: 0.54
Repetição 27 - Acurácia: 0.55, Precisão: 0.53, Recall: 0.93, F1-score: 0.67, ROC AUC: 0.55
Repetição 28 - Acurácia: 0.54, Precisão: 0.52, Recall: 0.92, F1-score: 0.67, ROC AUC: 0.54
Repetição 29 - Acurácia: 0.54, Precisão: 0.52, Recall: 0.92, F1-score: 0.67, ROC AUC: 0.54
Repetição 30 - Acurácia: 0.54, Precisão: 0.52, Recall: 0.93, F1-score: 0.67, ROC AUC: 0.54
Média das Acurácias: 0.54447 ± 0.00614
Média das Precisões: 0.52525 ± 0.00355
Média dos Re

In [ ]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all)
    
    # Inicializar o modelo MLP
    modelo_mlp = MLPClassifier(hidden_layer_sizes=(100), max_iter=1000)

    # Treinar o modelo no conjunto de treino
    modelo_mlp.fit(x_train, y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_mlp.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
Acurracy.append(media_acuracias)
Precision.append(media_precisoes)
Recall.append(media_recalls)
F1_score.append(media_f1_scores)
Roc_auc.append(media_roc_auc_scores)

AcurracyDP.append(dp_acuracias)
PrecisionDP.append(dp_precisoes)
RecallDP.append(dp_recalls)
F1_scoreDP.append(dp_f1_scores)
Roc_aucDP.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [00:30<14:47, 30.62s/it]

Repetição 1 - Acurácia: 0.98, Precisão: 0.82, Recall: 0.64, F1-score: 0.72, ROC AUC: 0.82


  7%|▋         | 2/30 [01:00<14:13, 30.47s/it]

Repetição 2 - Acurácia: 0.98, Precisão: 0.81, Recall: 0.62, F1-score: 0.70, ROC AUC: 0.81


 10%|█         | 3/30 [01:28<13:11, 29.30s/it]

...

Repetição 28 - Acurácia: 0.97, Precisão: 0.87, Recall: 0.49, F1-score: 0.63, ROC AUC: 0.75


 97%|█████████▋| 29/30 [16:27<00:32, 32.63s/it]

Repetição 29 - Acurácia: 0.98, Precisão: 0.81, Recall: 0.57, F1-score: 0.67, ROC AUC: 0.78


100%|██████████| 30/30 [17:11<00:00, 34.37s/it]

Repetição 30 - Acurácia: 0.98, Precisão: 0.80, Recall: 0.66, F1-score: 0.72, ROC AUC: 0.83
Média das Acurácias: 0.97645 ± 0.00177
Média das Precisões: 0.80596 ± 0.04730
Média dos Recalls: 0.60516 ± 0.04960
Média dos F1-scores: 0.68822 ± 0.02467
Média dos ROC AUC: 0.79916 ± 0.02374


In [ ]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all)
    
    #inicializar o modelo KNN
    modelo_knn = KNeighborsClassifier(n_neighbors=7)  # Escolha o número de vizinhos desejado

    # Treinar o modelo no conjunto de treino
    modelo_knn.fit(np.array(x_train), y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_knn.predict(np.array(x_test))

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
Acurracy.append(media_acuracias)
Precision.append(media_precisoes)
Recall.append(media_recalls)
F1_score.append(media_f1_scores)
Roc_auc.append(media_roc_auc_scores)

AcurracyDP.append(dp_acuracias)
PrecisionDP.append(dp_precisoes)
RecallDP.append(dp_recalls)
F1_scoreDP.append(dp_f1_scores)
Roc_aucDP.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [00:01<00:42,  1.48s/it]

Repetição 1 - Acurácia: 0.98, Precisão: 0.83, Recall: 0.75, F1-score: 0.79, ROC AUC: 0.87


  7%|▋         | 2/30 [00:02<00:38,  1.37s/it]

Repetição 2 - Acurácia: 0.98, Precisão: 0.81, Recall: 0.74, F1-score: 0.77, ROC AUC: 0.87


 10%|█         | 3/30 [00:04<00:36,  1.35s/it]

...

Repetição 28 - Acurácia: 0.98, Precisão: 0.79, Recall: 0.73, F1-score: 0.76, ROC AUC: 0.86


 97%|█████████▋| 29/30 [00:37<00:01,  1.30s/it]

Repetição 29 - Acurácia: 0.98, Precisão: 0.82, Recall: 0.74, F1-score: 0.78, ROC AUC: 0.87


100%|██████████| 30/30 [00:39<00:00,  1.32s/it]

Repetição 30 - Acurácia: 0.98, Precisão: 0.82, Recall: 0.76, F1-score: 0.79, ROC AUC: 0.87
Média das Acurácias: 0.98132 ± 0.00082
Média das Precisões: 0.81238 ± 0.01030
Média dos Recalls: 0.73690 ± 0.01586
Média dos F1-scores: 0.77271 ± 0.01106
Média dos ROC AUC: 0.86462 ± 0.00793


In [ ]:
import pandas as pd

# Definir os modelos
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']

# Criar um DataFrame do Pandas com os valores das variáveis
df = pd.DataFrame({
    'Modelos': Modelos,
    'Acuracia (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Acurracy, AcurracyDP)],
    'Precisão (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Precision, PrecisionDP)],
    'Recall (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Recall, RecallDP)],
    'F1 Score': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(F1_score, F1_scoreDP)],
    'AUC RoC': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Roc_auc, Roc_aucDP)]
})

# Função para destacar o maior valor de cada coluna
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

# Aplicar a função de destaque apenas para as colunas numéricas
numeric_columns = df.columns[1:]
styled_df = df.style.apply(highlight_max, subset=numeric_columns)

# Definir a cor do texto para a tabela
styled_df.set_properties(**{'color': 'white'}, subset=pd.IndexSlice[:, :])

# Exibir o DataFrame estilizado no Jupyter Notebook
styled_df


,Modelos,Acuracia (DREBIN),Precisão (DREBIN),Recall (DREBIN),F1 Score,AUC RoC
0,Decision Tree,0.9818 ± 0.0008,0.7715 ± 0.0123,0.8207 ± 0.0100,0.7953 ± 0.0080,0.9049 ± 0.0049
1,Naive Bayes,0.9569 ± 0.0000,0.0000 ± 0.0000,0.0000 ± 0.0000,0.0000 ± 0.0000,0.5000 ± 0.0000
2,MLP,0.9765 ± 0.0018,0.8060 ± 0.0473,0.6052 ± 0.0496,0.6882 ± 0.0247,0.7992 ± 0.0237
3,KNN,0.9813 ± 0.0008,0.8124 ± 0.0103,0.7369 ± 0.0159,0.7727 ± 0.0111,0.8646 ± 0.0079


### Bagging

In [ ]:
# Definir o classificador Decision Tree
modelo_arvore = DecisionTreeClassifier(criterion='gini')

# Definir o classificador de Bagging com Decision Tree como base
bagging_clf = BaggingClassifier(estimator = modelo_arvore, n_estimators=30)

# Treinar o modelo Bagging
bagging_clf.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = bagging_clf.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com Decision Tree:", accuracy_bagging)
print("Precisão do modelo Bagging com Decision Tree:", precision_bagging)
print("Recall do modelo Bagging com Decision Tree:", recall_bagging)
print("F1-score do modelo Bagging com Decision Tree:", f1_score_bagging)
print("AUC ROC do modelo Bagging com Decision Tree:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com Decision Tree:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBagging.append(accuracy_bagging)
PrecisionBagging.append(precision_bagging)
RecallBagging.append(recall_bagging)
F1Bagging.append(f1_score_bagging)
Roc_aucBagging.append(auc_roc_bagging)



Acurácia do modelo Bagging com Decision Tree: 0.9886834864163082
Precisão do modelo Bagging com Decision Tree: 0.9019607843137255
Recall do modelo Bagging com Decision Tree: 0.8273381294964028
F1-score do modelo Bagging com Decision Tree: 0.8630393996247655
AUC ROC do modelo Bagging com Decision Tree: 0.9116440353852757
Matriz de confusão do modelo Bagging com Decision Tree:
[[24591   100]
 [  192   920]]


In [8]:
# Inicializar o modelo de Naive Bayes com distribuição de Bernoulli
modelo_naive_bayes = BernoulliNB()

# Definir o classificador de Bagging com Naive Bayes como base
bagging_clf = BaggingClassifier(estimator=modelo_naive_bayes, n_estimators=30)

# Treinar o modelo Bagging
bagging_clf.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = bagging_clf.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com Naive Bayes:", accuracy_bagging)
print("Precisão do modelo Bagging com Naive Bayes:", precision_bagging)
print("Recall do modelo Bagging com Naive Bayes:", recall_bagging)
print("F1-score do modelo Bagging com Naive Bayes:", f1_score_bagging)
print("AUC ROC do modelo Bagging com Naive Bayes:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com Naive Bayes:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBagging.append(accuracy_bagging)
PrecisionBagging.append(precision_bagging)
RecallBagging.append(recall_bagging)
F1Bagging.append(f1_score_bagging)
Roc_aucBagging.append(auc_roc_bagging)

Acurácia do modelo Bagging com Naive Bayes: 0.5422661870503597
Precisão do modelo Bagging com Naive Bayes: 0.5238821138211383
Recall do modelo Bagging com Naive Bayes: 0.927158273381295
F1-score do modelo Bagging com Naive Bayes: 0.6694805194805195
AUC ROC do modelo Bagging com Naive Bayes: 0.5422661870503597
Matriz de confusão do modelo Bagging com Naive Bayes:
[[ 175  937]
 [  81 1031]]


In [ ]:
# Definir o classificador base (MLP)
modelo_mlp = MLPClassifier(hidden_layer_sizes=(100), max_iter=1000)

# Definir o classificador de Bagging com MLP como base
bagging_clf = BaggingClassifier(estimator=modelo_mlp, n_estimators=30)

# Treinar o modelo Bagging
bagging_clf.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = bagging_clf.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com MLP:", accuracy_bagging)
print("Precisão do modelo Bagging com MLP:", precision_bagging)
print("Recall do modelo Bagging com MLP:", recall_bagging)
print("F1-score do modelo Bagging com MLP:", f1_score_bagging)
print("AUC ROC do modelo Bagging com MLP:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com MLP:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBagging.append(accuracy_bagging)
PrecisionBagging.append(precision_bagging)
RecallBagging.append(recall_bagging)
F1Bagging.append(f1_score_bagging)
Roc_aucBagging.append(auc_roc_bagging)

Acurácia do modelo Bagging com MLP: 0.9807774289811262
Precisão do modelo Bagging com MLP: 0.8548387096774194
Recall do modelo Bagging com MLP: 0.6672661870503597
F1-score do modelo Bagging com MLP: 0.7494949494949495
AUC ROC do modelo Bagging com MLP: 0.8310815565278935
Matriz de confusão do modelo Bagging com MLP:
[[24565   126]
 [  370   742]]


In [ ]:
#inicializar o modelo KNN
modelo_knn = KNeighborsClassifier(n_neighbors=7)  # Escolha o número de vizinhos desejado

# Definir o classificador de Bagging com KNN como base
bagging_clf = BaggingClassifier(estimator=modelo_knn, n_estimators=30)

# Treinar o modelo Bagging
bagging_clf.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = bagging_clf.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com KNN:", accuracy_bagging)
print("Precisão do modelo Bagging com KNN:", precision_bagging)
print("Recall do modelo Bagging com KNN:", recall_bagging)
print("F1-score do modelo Bagging com KNN:", f1_score_bagging)
print("AUC ROC do modelo Bagging com KNN:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com KNN:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBagging.append(accuracy_bagging)
PrecisionBagging.append(precision_bagging)
RecallBagging.append(recall_bagging)
F1Bagging.append(f1_score_bagging)
Roc_aucBagging.append(auc_roc_bagging)

Acurácia do modelo Bagging com KNN: 0.9829089640739449
Precisão do modelo Bagging com KNN: 0.8273170731707317
Recall do modelo Bagging com KNN: 0.762589928057554
F1-score do modelo Bagging com KNN: 0.7936359382311652
AUC ROC do modelo Bagging com KNN: 0.8777106620563985
Matriz de confusão do modelo Bagging com KNN:
[[24514   177]
 [  264   848]]


In [ ]:
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']
# Criar um DataFrame do Pandas com os valores das variáveis
dfB = pd.DataFrame({'Modelos Bagging': Modelos, 'Acuracia (DREBIN)': AcurracyBagging,
                    'Precisão (DREBIN)': PrecisionBagging, 'Recall (DREBIN)':RecallBagging, 
                    'F1 Score': F1Bagging, 'AUC ROC': Roc_aucBagging})

print(tabulate(dfB, headers='keys', tablefmt='fancy_grid', floatfmt=".4f"))

╒════╤═══════════════════╤═════════════════════╤═════════════════════╤═══════════════════╤════════════╤═══════════╕
│    │ Modelos Bagging   │   Acuracia (DREBIN) │   Precisão (DREBIN) │   Recall (DREBIN) │   F1 Score │   AUC ROC │
╞════╪═══════════════════╪═════════════════════╪═════════════════════╪═══════════════════╪════════════╪═══════════╡
│  0 │ Decision Tree     │              0.9887 │              0.9020 │            0.8273 │     0.8630 │    0.9116 │
├────┼───────────────────┼─────────────────────┼─────────────────────┼───────────────────┼────────────┼───────────┤
│  1 │ Naive Bayes       │              0.9569 │              0.0000 │            0.0000 │     0.0000 │    0.5000 │
├────┼───────────────────┼─────────────────────┼─────────────────────┼───────────────────┼────────────┼───────────┤
│  2 │ MLP               │              0.9808 │              0.8548 │            0.6673 │     0.7495 │    0.8311 │
├────┼───────────────────┼─────────────────────┼─────────────────────┼──

### Balanceamento de dados

In [ ]:
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(x_all, y_all)

x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(197524, 8) (197524,)
(49382, 8) (49382,)


In [ ]:
# Calcular proporção das classes em y_all
unique_classes, class_counts = np.unique(y_all, return_counts=True)
proporcao_classes_resampled = class_counts / len(y_all)

print("Proporção das classes no conjunto original:")
for classe, proporcao in zip(unique_classes, proporcao_classes_resampled):
    print(f"Classe {classe}: {proporcao:.2f}")

# Calcular proporção das classes em y_resampled
unique_classes, class_counts = np.unique(y_resampled, return_counts=True)
proporcao_classes_resampled = class_counts / len(y_resampled)

print("Proporção das classes no conjunto resampleado:")
for classe, proporcao in zip(unique_classes, proporcao_classes_resampled):
    print(f"Classe {classe}: {proporcao:.2f}")

Proporção das classes no conjunto original:
Classe 0: 0.96
Classe 1: 0.04
Proporção das classes no conjunto resampleado:
Classe 0: 0.50
Classe 1: 0.50


### Modelos Base após o Balanceamento

In [ ]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):
    x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled)
    # Inicializar o modelo de árvore de decisão com Gini Impurity e sem limitação de altura
    modelo_arvore = DecisionTreeClassifier(criterion='gini')

    # Treinar o modelo no conjunto de treino
    modelo_arvore.fit(x_train, y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_arvore.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
AcurracyPosBal.append(media_acuracias)
PrecisionPosBal.append(media_precisoes)
RecallPosBal.append(media_recalls)
F1_scorePosBal.append(media_f1_scores)
Roc_aucPosBal.append(media_roc_auc_scores)

AcurracyDPPosBal.append(dp_acuracias)
PrecisionDPPosBal.append(dp_precisoes)
RecallDPPosBal.append(dp_recalls)
F1_scoreDPPosBal.append(dp_f1_scores)
Roc_aucDPPosBal.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [00:00<00:15,  1.93it/s]

Repetição 1 - Acurácia: 0.98, Precisão: 0.97, Recall: 0.99, F1-score: 0.98, ROC AUC: 0.98


  7%|▋         | 2/30 [00:01<00:14,  1.93it/s]

Repetição 2 - Acurácia: 0.98, Precisão: 0.97, Recall: 0.99, F1-score: 0.98, ROC AUC: 0.98


 10%|█         | 3/30 [00:01<00:14,  1.92it/s]

...

Repetição 28 - Acurácia: 0.98, Precisão: 0.97, Recall: 0.99, F1-score: 0.98, ROC AUC: 0.98


 97%|█████████▋| 29/30 [00:14<00:00,  1.95it/s]

Repetição 29 - Acurácia: 0.98, Precisão: 0.98, Recall: 0.99, F1-score: 0.98, ROC AUC: 0.98


100%|██████████| 30/30 [00:15<00:00,  1.94it/s]

Repetição 30 - Acurácia: 0.98, Precisão: 0.97, Recall: 0.99, F1-score: 0.98, ROC AUC: 0.98
Média das Acurácias: 0.98132 ± 0.00053
Média das Precisões: 0.97398 ± 0.00101
Média dos Recalls: 0.98906 ± 0.00071
Média dos F1-scores: 0.98146 ± 0.00053
Média dos ROC AUC: 0.98132 ± 0.00053


In [ ]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled)
    
    #Inicializar o modelo de Naive Bayes com distribuição de Bernoulli
    modelo_naive_bayes = BernoulliNB()

    # Treinar o modelo no conjunto de treino
    modelo_naive_bayes.fit(x_train, y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_naive_bayes.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
Acurracy.append(media_acuracias)
Precision.append(media_precisoes)
Recall.append(media_recalls)
F1_score.append(media_f1_scores)
Roc_auc.append(media_roc_auc_scores)

AcurracyDP.append(dp_acuracias)
PrecisionDP.append(dp_precisoes)
RecallDP.append(dp_recalls)
F1_scoreDP.append(dp_f1_scores)
Roc_aucDP.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  7%|▋         | 2/30 [00:00<00:03,  7.27it/s]

Repetição 1 - Acurácia: 0.55, Precisão: 0.53, Recall: 0.93, F1-score: 0.67, ROC AUC: 0.55
Repetição 2 - Acurácia: 0.55, Precisão: 0.53, Recall: 0.92, F1-score: 0.67, ROC AUC: 0.55


 13%|█▎        | 4/30 [00:00<00:03,  7.34it/s]

Repetição 3 - Acurácia: 0.55, Precisão: 0.53, Recall: 0.93, F1-score: 0.67, ROC AUC: 0.55
Repetição 4 - Acurácia: 0.54, Precisão: 0.53, Recall: 0.92, F1-score: 0.67, ROC AUC: 0.54


 20%|██        | 6/30 [00:00<00:03,  7.57it/s]

...

Repetição 25 - Acurácia: 0.54, Precisão: 0.52, Recall: 0.93, F1-score: 0.67, ROC AUC: 0.54
Repetição 26 - Acurácia: 0.54, Precisão: 0.53, Recall: 0.93, F1-score: 0.67, ROC AUC: 0.54


 93%|█████████▎| 28/30 [00:03<00:00,  7.29it/s]

Repetição 27 - Acurácia: 0.54, Precisão: 0.52, Recall: 0.93, F1-score: 0.67, ROC AUC: 0.54
Repetição 28 - Acurácia: 0.55, Precisão: 0.53, Recall: 0.93, F1-score: 0.67, ROC AUC: 0.55


100%|██████████| 30/30 [00:04<00:00,  7.22it/s]

Repetição 29 - Acurácia: 0.54, Precisão: 0.53, Recall: 0.92, F1-score: 0.67, ROC AUC: 0.54
Repetição 30 - Acurácia: 0.54, Precisão: 0.53, Recall: 0.92, F1-score: 0.67, ROC AUC: 0.54
Média das Acurácias: 0.54536 ± 0.00131
Média das Precisões: 0.52577 ± 0.00076
Média dos Recalls: 0.92546 ± 0.00120
Média dos F1-scores: 0.67057 ± 0.00084
Média dos ROC AUC: 0.54536 ± 0.00131


### Resultados